**TENSORFLOW**

In [ ]:
! pip -q install imageio
! pip -q install scikit-image
! pip install -q git+https://github.com/tensorflow/docs

**Mounting google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
projectPath = r'drive/MyDrive'
folderCeleb = os.path.join(projectPath, 'celeba')

**importing library and define interpolation function**

In [ ]:
from absl import logging

import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf.random.set_seed(0)

import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time

try:
  from google.colab import files
except ImportError:
  pass

from IPython import display
from skimage import transform

latent_dim = 128

def interpolate_hypersphere(v1, v2, num_steps):
  v1_norm = tf.norm(v1)
  v2_norm = tf.norm(v2)
  v2_normalized = v2 * (v1_norm / v2_norm)

  vectors = []
  for step in range(num_steps):
    interpolated = v1 + (v2_normalized - v1) * step / (num_steps - 1)
    interpolated_norm = tf.norm(interpolated)
    interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
    vectors.append(interpolated_normalized)
  return tf.stack(vectors)

# Simple way to display an image.
def display_image(image):
  image = tf.constant(image)
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return PIL.Image.fromarray(image.numpy())

# Given a set of images, show an animation.
def animate(images):
  images = np.array(images)
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images)
  return embed.embed_file('./animation.gif')

logging.set_verbosity(logging.ERROR)

**Set workspace environment**

In [ ]:
data = os.path.join(folderCeleb, 'img_align_celeba')
getData = os.listdir(data)[0:1000]
getSample = os.path.join(data, getData[100])

**plot random image file**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(getSample)
imgplot = plt.imshow(img)
plt.show()

**create normalize arr_image and normalize tensorflow dataset**

In [ ]:
def my_func(arg):
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  return arg

arr_image = []
tensor_dataset = []
for i in range(0, 100):
  getSample = os.path.join(data, getData[i])
  an_image = PIL.Image.open(getSample)

  # tf input
  image_array = np.asarray(an_image)
  normalize = [255]
  normalzaArray = np.divide(image_array, normalize)
  arr_image.append(normalzaArray)

  # tf output
  resize_image = an_image.resize((128,128))
  image_array = np.asarray(resize_image)
  normalize = [255]
  normalzaArray = np.divide(image_array, normalize)
  change2tensor = my_func(normalzaArray)
  tensor_dataset.append(change2tensor)  

**plot 100 images**

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

In [ ]:
fig = plt.figure(figsize=(20., 20.))
grid = ImageGrid(fig, 111, 
                 nrows_ncols=(10, 10),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes
                 )

for ax, im in zip(grid, arr_image):
    ax.imshow(im)

plt.show()

**interpolate vectors**

In [ ]:
def interpolate_between_vectors():
  v1 = tf.random.normal([latent_dim])
  v2 = tf.random.normal([latent_dim])
  vectors = interpolate_hypersphere(v1, v2, 128)

  interpolated_images = tensor_dataset

  return interpolated_images

interpolated_images = interpolate_between_vectors()
animate(interpolated_images)

**set target image**

In [ ]:
image_from_module_space = True  # @param { isTemplate:true, type:"boolean" }

def get_module_space_image():
  vector = tf.random.normal([1, latent_dim])
  images = tensor_dataset[0]
  return images

def upload_image():
  uploaded = files.upload()
  image = imageio.imread(uploaded[list(uploaded.keys())[0]])
  return transform.resize(image, [128, 128])

if image_from_module_space:
  target_image = get_module_space_image()
else:
  target_image = upload_image()

display_image(target_image)

In [ ]:
tf.random.set_seed(42)
initial_vector = tf.random.normal([1, latent_dim])

**create loss function**

In [ ]:
def find_closest_latent_vector(initial_vector, num_optimization_steps,
                               steps_per_image):
  images = []
  losses = []

  vector = tf.Variable(initial_vector)  
  optimizer = tf.optimizers.Adam(learning_rate=0.01)
  loss_fn = tf.losses.MeanAbsoluteError(reduction="sum")

  for step in range(num_optimization_steps):
    if (step % 100)==0:
      print()
    print('.', end='')
    with tf.GradientTape() as tape:
      image = tensor_dataset[0]
      if (step % steps_per_image) == 0:
        images.append(image.numpy())
      target_image_difference = loss_fn(image, target_image[:,:,:3])
      regularizer = tf.abs(tf.norm(vector) - np.sqrt(latent_dim))

      loss = target_image_difference + regularizer
      losses.append(loss.numpy())
    grads = tape.gradient(loss, [vector])
    optimizer.apply_gradients(zip(grads, [vector]))

  return images, losses


num_optimization_steps=200
steps_per_image=5
images, loss = find_closest_latent_vector(initial_vector, num_optimization_steps, steps_per_image)

**plot loss function**

In [ ]:
plt.plot(loss)
plt.ylim([0,max(plt.ylim())])

**Displaying different**

In [ ]:
display_image(np.concatenate([images[-1], target_image], axis=1))